<a href="https://colab.research.google.com/github/habibsifat/BengaliDictionary/blob/master/Word_Pre_trained_w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from __future__ import print_function
__author__ = 'maxim'
import numpy as np
import pandas as pd
import gensim
import string
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file

In [0]:
!ls 'drive/My Drive/Colab Notebooks/Capstone/Lenth-5.txt'

ls: cannot access 'drive/My Drive/Colab Notebooks/Capstone/Lenth-5.txt': No such file or directory


In [0]:
corpus_file = 'drive/My Drive/Colab Notebooks/Capstone/Lenth-5.txt'

In [0]:
print('\nPreparing the sentences...')
#max_sentence_len = 40

sentences = []
with open(corpus_file, 'r', encoding="utf-8-sig") as f:
    for line in f:
        sentences.append(line.split())
        
print('Num sentences:', len(sentences))


Preparing the sentences...
Num sentences: 4626


# W2V

In [0]:
#Training word2vec 

print('\nTraining word2vec...')
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=5)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
#print('Checking similar words:')
#for word in ['৬ষ্ঠ', 'ঘুরে', 'সময়ই', 'করানো']:
 # most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
#print('  %s -> %s' % (word, most_similar))

In [0]:
word_model.wv.vocab

In [0]:
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

In [0]:
word = "কার্তিক"  # for any word in model
i = word_model.wv.vocab[word].index
print(i)
print(word_model.wv.index2word[18])
if word_model.wv.index2word[i] == word :
  # will be true
  print("True")

In [0]:
#print(word_model.wv.vocab["ও"].index)
print(word_model.wv.index2word[869])

ইউনিয়ন


# Data Load

In [0]:
print('\nPreparing the sentences...')
corpus_file = 'drive/My Drive/Colab Notebooks/Capstone/Lenth-5.txt'
max_sentence_len = 5

sentences = []
with open(corpus_file, 'r', encoding="utf-8-sig") as f:
    for line in f:
        sentences.append(line.split())
        
print('Num sentences:', len(sentences))


Preparing the sentences...
Num sentences: 4626


In [0]:
print('\nPreparing the data for LSTM...')
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (4626, 5)
train_y shape: (4626,)


# Train

In [0]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=emdedding_size, return_sequences=False))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Model Predict


In [0]:
#def sample(preds, temperature=1.0):
  if temperature <= 0:
    #return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  print("Preds : ",preds)
  #probas = np.random.multinomial(1, preds, 1)
  #return np.argmax(probas)
  #return np.argmax(preds)

In [0]:
def generate_next(text, num_generated=1):
  word_idxs = [word2idx(word) for word in text.split()]
  print(word_idxs)
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs)) #print dimension
    print("prediction:",end='')
    print(prediction[-1])
    print(np.shape(prediction))
    maxx=np.amax(prediction[-1])
    print("Max : ",maxx)
    print("Sum ",np.sum(prediction[-1]))
    #idx = sample(prediction[-1], temperature=0.7)
    j=0
    for a in prediction[-1]:
      #print("Value of ",end='')
      #print(j,end='')
      if a==maxx:
        print("Max : ",a)
        idx=j
      j=j+1
      #print()
    #idx=np.argmax(prediction[-1])
    print("idx:",end='')
    print(idx)
    word_idxs.append(idx)
    print(word_idxs)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [0]:
def on_epoch_end(epoch, _):
  print('\n For Lenth-5 Generating text after epoch: %d' % epoch)
  texts = [
    '* তড়িৎ প্রকৌশল বলতে ',
      '* এটি বাংলাদেশে বাংলা '
  ]
  for text in texts:
    sample = generate_next(text)
    print('%s -> %s' % (text, sample))

model.fit(train_x, train_y,
          batch_size=128,
          epochs=1,
callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

In [0]:
#Partly train model
#model.fit(first_training, first_classes, batch_size=32, nb_epoch=20)

#Save partly trained model
model.save('word_generate.h5')

#Load partly trained model
from keras.models import load_model
model = load_model('word_generate.h5')

#Continue training
#model.fit(second_training, second_classes, batch_size=32, nb_epoch=20)

In [0]:
def generate_next(text, num_generated=1):
  word_idxs = [word2idx(word) for word in text.split()]
  #print(word_idxs)
  Str1 = "অনেক"
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs)) #print dimension
    max_word = prediction[-1].argsort()[-10:][::-1]
    #print(max_word)
    j=1
    for idx in max_word:
      word_idxs.append(idx)
      Distance = levenshtein_ratio_and_distance(Str,idx)
      Ratio = levenshtein_ratio_and_distance(Str1,Str2,ratio_calc = True)
      print("Max word %d : %s " % (j,idx2word(idx)))
      print("Distance : %s And Ratio : %s " % (Distance,Ratio)
        #j=j+1
      #print(word_idxs)
return ' '.join(idx2word(idx) for idx in word_idxs)